<a href="https://colab.research.google.com/github/BaudraccoTomas/ChatBootInteligente_Alura/blob/main/Chatbot_Inteligente_Alura_Sprint2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Configurar ambiente

In [1]:
!python -m spacy download es_core_news_md
!pip install jellyfish
!pip install transformers
!pip install python-docx

2023-10-09 12:10:16.110121: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 12:10:18.230401: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 9.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.6 MB

In [2]:
#Instalando bibliotecas
import pandas as pd
import re, os, random, pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import jellyfish
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch
from docx import Document
import csv
import nltk
nltk.download('punkt')

#Definiendo variables del proyecto:
nlp = spacy.load('es_core_news_md')

#Conectando al Google Drive
from google.colab import drive
drive.mount('/content/drive')
folder = '/content/drive/MyDrive/Chatbot'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive


# 2. Importar verbos

In [3]:
# Importar la lista_verbos:
pickle_file = open("/content/drive/MyDrive/Chatbot/verbos/lista_verbos.pickle", "rb")
lista_verbos = pickle.load(pickle_file)

# Importar el diccionario:
pickle_file = open("/content/drive/MyDrive/Chatbot/verbos/verbos_irregulares.pickle", "rb")
verbos_irregulares = pickle.load(pickle_file)

#3. Tratamiento de datos

In [44]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.2 MB/s eta 0:00:00


In [45]:
from unidecode import unidecode
from nltk.tokenize import WhitespaceTokenizer

In [52]:
#Función para encontrar la raiz de las palabras
def raiz(palabra):
  """
  Funcion para comparar similitud entre palabras\n
  recibe(palabraAComparar, listaDePalabrasdeDondeComparar)\n
  return (palabra elegida,\n
          porcentaje de similitud,\n
          palabra mas cercana)
  """
  similar = 0
  contador = 0
  verbo_similar = ""
  for verbo in lista_verbos:
    contador += 1
    radio = jellyfish.jaro_winkler_similarity(palabra, verbo)
    if radio > similar:
      similar = radio
      verbo_similar = verbo
  if similar < 0.93:
    verbo_final = palabra
  else:
    verbo_final = verbo_similar
  return verbo_final

def tratamiento_texto(texto):
  """
  Se ingresa un texto y se devuelve texto:\n
  · En minuscula\n
  · Sin acentos\n
  · Sin signos de puntuacion
  """
  lista_palabras = []
  #texto = texto.lower()
  texto = unidecode(texto)
  espaciado = WhitespaceTokenizer()
  texto_espacios = espaciado.tokenize(texto)

  for palabra in texto_espacios:
    palabra_limpia = "".join([letra for letra in palabra if letra.isalnum()])
    lista_palabras.append(palabra_limpia)

  texto_limpio = " ".join([i for i in lista_palabras])
  return texto_limpio

#Función para reemplazar el final de una palabra por 'r'
def reemplazar_terminacion(palabra):
  """
  Intenta acercar los verbos conjugados lo mas posible a su forma en infinitivo
  · Remplaza las terminaciones "ar", "es", "me", "as", "te" por "r" y la terminacion "ste" por "ar"
  """
  lista_palabras = []
  espaciado = WhitespaceTokenizer()
  texto_espacios = espaciado.tokenize(palabra)

  for texto in texto_espacios:
    if texto[-3:] == "ste" and texto != "este":
      texto = texto[:-3] + "ar"
    elif (texto[-2:] == "es" or texto[-2:] == "me" or texto[-2:] == "as" or texto[-2:] == "te") and len(texto) > 3:
      texto = texto[:-2] + "r"
    lista_palabras.append(texto)

  texto_limpio = " ".join([i for i in lista_palabras])
  return texto_limpio.split()[0]

#Función para adicionar o eliminar tokens
def revisar_tokens(texto, tokens):
    # Si la lista de tokens está vacía, vamos a incluir tokens especiales
    if len(tokens) == 0:
        # Si encuentras alguna palabra compuesta especial en el texto, agrega su token
        if 'cientifico de datos' in texto or 'data scientist' in texto:
            tokens.append('datascientist')
        if 'ciencia de datos' in texto or 'data science' in texto:
            tokens.append('datascience')
        if 'elprofealejo' in texto or 'el profe alejo' in texto or 'profe alejo' in texto or 'profealejo' in texto:
            tokens.append('elprofealejo')
    else:
        # Tokens a eliminar si se encuentran en el texto
        elementos_a_eliminar = ["cual", "que", "quien", "cuanto", "cuando", "como"]
        if 'hablame' in texto and 'hablar' in tokens:
            elementos_a_eliminar.append('hablar')
        elif 'cuentame' in texto and 'contar' in tokens:
            elementos_a_eliminar.append('contar')
        elif 'hago' in texto and 'hacer' in tokens:
            elementos_a_eliminar.append('hacer')
        elif 'entiendes' in texto and 'entender' in tokens:
            elementos_a_eliminar.append('entender')
        elif 'sabes' in texto and 'saber' in tokens:
            elementos_a_eliminar.append('saber')

        # Elimina los elementos de la lista de tokens si están en la lista de elementos a eliminar
        tokens = [token for token in tokens if token not in elementos_a_eliminar]

    return tokens


#Función para devolver los tokens normalizados del texto
def normalizar(texto):
  tokens=[]
  palabras_normalizadas = []
  tokens=revisar_tokens(tratamiento_texto(texto), tokens)
  doc = nlp(texto)
  for t in doc:
    if t.is_alpha:
      palabra = t.text.lower()
      # Reemplazar verbos irregulares
      lemma = verbos_irregulares.get(palabra, t.lemma_)
      # Eliminar caracteres especiales
      lemma = re.sub(r'[^\w\s+\-*/]', '', lemma)
      # Agregar la palabra normalizada a la lista
      palabras_normalizadas.append(lemma)
      if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
        if t.pos_=='VERB':
          lemma = reemplazar_terminacion(lemma)
          tokens.append(raiz(tratamiento_texto(lemma)))
        else:
          tokens.append(tratamiento_texto(lemma))

  tokens = list(dict.fromkeys(tokens))
  tokens = list(filter(None, tokens))
  tokens = revisar_tokens(tratamiento_texto(texto), tokens)
  return tokens

# 4. Cargar bases de documentos

In [19]:
#Importando bases de dialogo fluído
txt_folder_path = "/content/drive/MyDrive/Chatbot/dialogos"
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
lista_dialogos, lista_dialogos_respuesta, lista_tipo_dialogo = [],[],[]
for idx in range(len(lista_documentos)):
  f=open(txt_folder_path+'/'+lista_documentos[idx], 'r', encoding='utf-8', errors='ignore')
  contador = 0
  for linea in f.read().split('\n'):
    if contador % 2 == 0:
      #linea = normalizacion(linea)
      linea = re.sub(r"[^\w\s+\-*/]", '', linea)
      linea = re.sub(r"[.*^\n]", '', linea)
      lista_dialogos.append(linea)
      lista_tipo_dialogo.append(lista_documentos[idx][:-4])
    else:
      linea = re.sub(r"[^\w\s+\-*/]", '', linea)
      linea = re.sub(r"[.*^\n]", '', linea)
      lista_dialogos_respuesta.append(linea)
    contador += 1

#Creando Dataframe de diálogos
datos = {'dialogo':lista_dialogos,'respuesta':lista_dialogos_respuesta,'tipo':lista_tipo_dialogo,'interseccion':0,'jaro_winkler':0,'probabilidad':0}
df_dialogo = pd.DataFrame(datos)
df_dialogo = df_dialogo.drop_duplicates(keep='first')
df_dialogo.reset_index(drop=True, inplace=True)


In [ ]:
df_dialogo

In [ ]:
#Importando bases csv
txt_folder_path = '/content/drive/MyDrive/Chatbot/documentos'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".csv")]
documento_csv = 'tu código aqui'

#Importando bases csv
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".csv")]
documento_csv = ''
for i in range(len(lista_documentos)):
  with open(txt_folder_path+'/'+lista_documentos[i], "r", encoding="utf-8") as csv_txt:
    csv_text = csv.reader(csv_txt)
    for fila in csv_text:
      print(fila[-1])
      if fila[-1]!='frase':
        documento_csv += fila[-1]
print(documento_csv)

In [ ]:
#Importando bases docx

txt_folder_path = '/content/drive/MyDrive/Chatbot/documentos'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".docx")]
documento_docx = ''
for i in range(len(lista_documentos)):
  for t in Document(txt_folder_path+'/'+lista_documentos[i]).paragraphs:
    documento_docx += t.text.replace('*','\n\n*')+ "\n"
print(documento_docx)

In [ ]:
#Importando bases txt
txt_folder_path = '/content/drive/MyDrive/Chatbot/documentos'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
documento_txt = ''
for i in range(len(lista_documentos)):
  with open(txt_folder_path+'/'+lista_documentos[i], "r", encoding="utf-8") as txt:
    txt_new = txt.read()
    for i in txt_new:
      documento_txt += i
print(documento_txt)

In [53]:
documento = documento_csv + documento_txt + documento_docx
lista_frases = nltk.sent_tokenize(documento,'spanish')
lista_frases_normalizadas = [' '.join(normalizar(x)) for x in lista_frases]

# 5. Buscar respuesta del Chatbot

In [54]:
def interseccion(usuario, pregunta_dialogo):
  interseccion = 0
  largo_palabra = 0
  dialogo = WhitespaceTokenizer().tokenize(pregunta_dialogo)
  for palabra_usuario in usuario:
    largo_palabra += 1
    for palabra_dialogo in dialogo:
      if palabra_usuario == palabra_dialogo:
        interseccion += 1
  porcentaje = interseccion / largo_palabra
  return round(porcentaje, 2)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def similarity(pregunta_usuario, pregunta_dialogo):
  vectorizer = TfidfVectorizer()
  vectorizer.fit_transform(df_dialogo["dialogo"])
  text1 = vectorizer.transform([pregunta_usuario])
  text2 = vectorizer.transform([pregunta_dialogo])
  return cosine_similarity(text1, text2)

In [59]:
#Función para verificar si el usuário inició un diálogo
def dialogo(user_response):
  user_response = tratamiento_texto(user_response) #Tratando el texto
  user_response = re.sub(r"[^\w\s]", '', user_response) #Elimina signos de puntuación
  consulta = " ".join(user_response)
  df = df_dialogo.copy()
  for idx,row in df.iterrows():
    df.at[idx,'interseccion'] = interseccion(user_response, row["dialogo"])
    df.at[idx,'similarity'] = similarity(consulta, row["dialogo"])[0][0]
    df.at[idx,'jaro_winkler'] = jellyfish.jaro_winkler_similarity(consulta, row["dialogo"])
    df.at[idx,'probabilidad'] = max(df.at[idx,'interseccion'],df.at[idx,'similarity'],df.at[idx,'jaro_winkler'])
  df.sort_values(by=['probabilidad','jaro_winkler'], inplace=True, ascending=False)
  probabilidad = df['probabilidad'].head(1).values[0]
  if probabilidad >= 0.93:
    print('Respuesta encontrada por el método de comparación de textos - Probabilidad: ', probabilidad)
    respuesta = df['respuesta'].head(1).values[0]
  else:
    respuesta = ''
  return respuesta


In [60]:
#Cargar tu modelo entrenado aqui(recuerda siempre cargar el modelo y el vectorizer o tokenizer usado en el entrenamiento del modelo):
ruta_modelo = '/content/drive/MyDrive/Chatbot/modelo'
Modelo_TF = BertForSequenceClassification.from_pretrained(ruta_modelo)
tokenizer_TF = BertTokenizer.from_pretrained(ruta_modelo)


In [114]:
def clase_encontrada(pregunta):
  frase = ' '.join(normalizar(pregunta))
    # Tokenizar la frase de entrada
  tokens = tokenizer_TF.encode_plus(
      frase,
      add_special_tokens=True,
      max_length=128,
      padding='max_length',
      truncation=True,
      return_tensors='pt'
  )

  # Obtener los input_ids y attention_mask
  input_ids = tokens['input_ids']
  attention_mask = tokens['attention_mask']

  # Realizar la predicción
  with torch.no_grad():
      outputs = Modelo_TF(input_ids, attention_mask)

  # Obtener las etiquetas predichas
  etiquetas_predichas = torch.argmax(outputs.logits, dim=1)

  # Decodificar las etiquetas predichas
  etiquetas_decodificadas = etiquetas_predichas.tolist()
  diccionario = {14: 'Sentimiento', 13: 'Saludos', 10: 'Nombre', 9: 'Identidad', 6: 'ElProfeAlejo', 1: 'Aprendizaje', 8: 'Funcion', 15: 'Usuario', 11: 'Origen', 5: 'Edad', 0: 'Agradecimiento', 3: 'Continuacion', 2: 'Contacto', 4: 'Despedida', 12: 'Otros', 7: 'Error'}
  llave_buscada = etiquetas_decodificadas[0]
  clase_encontrada = diccionario[llave_buscada]

  return clase_encontrada

def clasificacion_modelo(pregunta):

  clase_encontrada1 = clase_encontrada(pregunta)

  #Buscar respuesta más parecida en la clase encontrada
  df = df_dialogo[df_dialogo['tipo'] == clase_encontrada1]
  df.reset_index(inplace=True)
  vectorizer = TfidfVectorizer()
  dialogos_num = vectorizer.fit_transform(df['dialogo'])
  pregunta_num = vectorizer.transform([tratamiento_texto(pregunta)])
  similarity_scores = cosine_similarity(dialogos_num, pregunta_num)
  indice_pregunta_proxima = similarity_scores.argmax()

  if max(similarity_scores)>0.5 and clase_encontrada1 not in ['Otros']:
    #print('Respuesta encontrada por el modelo Transformers - tipo:',clase_encontrada1)
    respuesta = df['respuesta'][indice_pregunta_proxima]
  else:
    respuesta = ''
  return respuesta


  #Buscar respuesta más parecida en la clase encontrada
  df = df_dialogo[df_dialogo['tipo'] == clase_encontrada]
  df.reset_index(inplace=True)
  vectorizer = TfidfVectorizer()
  dialogos_num = vectorizer.fit_transform(df['dialogo'])
  pregunta_num = vectorizer.transform([tratamiento_texto(pregunta)])
  similarity_scores = cosine_similarity(dialogos_num, pregunta_num)
  indice_pregunta_proxima = similarity_scores.argmax()

  if max(similarity_scores)>0.5 and clase_encontrada not in ['Otros']:
    print('Respuesta encontrada por el modelo Transformers - tipo:',clase_encontrada)
    respuesta = df['respuesta'][indice_pregunta_proxima]
  else:
    respuesta = ''
  return respuesta

#Función para devolver la respuesta de los documentos
def respuesta_documento(pregunta):
  pregunta = normalizar(pregunta)
  def contar_coincidencias(frase):
    return sum(1 for elemento in pregunta if elemento in frase)

  diccionario = {valor: posicion for posicion, valor in enumerate(lista_frases_normalizadas)}
  lista = sorted(list(diccionario.keys()), key=contar_coincidencias, reverse=True)[:6]
    #Hasta aqui ya tengo mi lista con las 6 respuestas con mayor coincidencia de tokens
  #Convierte la pregunta en frase
  #Adiciona la frase convertida al final de la lista
  #Inicializa un TfidfVectorizer utilizando la función normalizar como tokenizer
  #Entrenalo con fit_transform sobre la lista y asigna el resultado a la variable tfidf
  lista.append(''.join(pregunta))
  for elemento in lista:
    print(elemento)
  TfidfVec = TfidfVectorizer(tokenizer=normalizar, token_pattern=None)
  tfidf = TfidfVec.fit_transform(lista)

  #Aplica cosine_similarity entre el último elemento de tfidf y todos los elementos de tfidf y guárdalo en vals
  vals = cosine_similarity(tfidf[-1], tfidf)
  print(vals)
  idx = vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  req_tfidf = round(flat[-2],2)
  if req_tfidf>=0.22:
    print('Respuesta encontrada por el método TfidfVectorizer - Probabilidad:', req_tfidf)
    respuesta = lista_frases[diccionario[lista[idx]]]
  else:
    respuesta = ''
  return respuesta

#Función para devolver una respuesta final buscada en todos los métodos disponibles
def respuesta_chatbot(pregunta):
  respuesta = respuesta_documento(pregunta)
  if respuesta != '':
    return respuesta
  else:
    respuesta = dialogo(pregunta)
    if respuesta != '':
      return respuesta
    else:
      respuesta = clasificacion_modelo(pregunta)
      if respuesta != '':
        return respuesta
      else:
        return 'Respuesta no encontrada'

In [115]:
pregunta='ciencia de datos'
respuesta_documento(pregunta)

datascience ciencia dato ser campo academico interdisciplinario utilizar estadistico computacion cientifico metodo proceso algoritmo sistema obtener recolectar extraer tratar analizar presentar informe partir ruidoso estructurado no
datascience ciencia dato ser multifacetico poder describir paradigma investigacion metodo disciplina flujo trabajo profesion
datascience ciencia dato integrar conocimiento dominio aplicacion subyacente ejemplo economico finanza medicina natural tecnologia informacion ser concepto consistente unificar estadistico analisis informatica metodo relacionado comprender analizar fenomeno real
datascience ciencia dato utilizar metodo tecnica teoria extraido campo dentro contexto matematica estadistica computacion informacion conocimiento dominio
datascience ciencia dato ser diferente informatica estadistica informacion ganador premio Turing Jim Gray imaginar cuarto paradigma empirico teorico computacional ahora basado afirmar todo estar cambiar debido impacto tecnol

''

# 6. Ejecutar Chatbot

In [116]:
pregunta='hola como estas mi hermano?'
respuesta = respuesta_chatbot(pregunta)
print(respuesta)

datascience ciencia dato ser diferente informatica estadistica informacion ganador premio Turing Jim Gray imaginar cuarto paradigma empirico teorico computacional ahora basado afirmar todo estar cambiar debido impacto tecnologia avalancha
investigador Yangyong Zhu Yun Xiong Research Center for Dataology and Data Science publicar Introduction to donde manifestar diferencia ciencia natural social Datologia Ciencia Datos tomar dato red objeto estudio ser lanzar IEEE Task Force on Advanced Analytics primero conferencia internacional International Conference
datascientist datascience roadmap convertir cientifico dato existir forma adquirir conocimiento necesario universidad estar empezar ofrecer curso diplomado alguno maestria doctorado ciencia
IBM Network Skills ofrecer Certificado Profesional Ciencia Datos tener costar el poder pedir ayuda economico curar gratuitamente estar compuesto curso duracion mes mas informacion
ejemplo aplicacion Ciencia Datos Marketing septiembre BusinessWeek pub

In [79]:
pregunta='inteligencia artificial?'
respuesta = respuesta_chatbot(pregunta)
print(respuesta)

Respuesta clasificion de modelos
Respuesta no encontrada


In [81]:
pregunta='cuantos idiomas habla el profe alejandro?'
respuesta = respuesta_chatbot(pregunta)
print(respuesta)

Respuesta clasificion de modelos
Respuesta no encontrada
